In [91]:
import sqlite3

In [93]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [95]:
%sql sqlite:///sql-murder-mystery.db


In [97]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///sql-murder-mystery.db
Done.


name
crime_scene_report
drivers_license
person
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
income
solution


In [99]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///sql-murder-mystery.db
Done.


type,name,tbl_name,rootpage,sql
table,crime_scene_report,crime_scene_report,2,"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"
table,drivers_license,drivers_license,3,"CREATE TABLE drivers_license ( id integer PRIMARY KEY, age integer, height integer, eye_color text, hair_color text, gender text, plate_number text, car_make text, car_model text )"
table,person,person,4,"CREATE TABLE person ( id integer PRIMARY KEY, name text, license_id integer, address_number integer, address_street_name text, ssn integer, FOREIGN KEY (license_id) REFERENCES drivers_license(id) )"
table,facebook_event_checkin,facebook_event_checkin,5,"CREATE TABLE facebook_event_checkin ( person_id integer, event_id integer, event_name text, date integer, FOREIGN KEY (person_id) REFERENCES person(id) )"
table,interview,interview,6,"CREATE TABLE interview ( person_id integer, transcript text, FOREIGN KEY (person_id) REFERENCES person(id) )"
table,get_fit_now_member,get_fit_now_member,7,"CREATE TABLE get_fit_now_member ( id text PRIMARY KEY, person_id integer, name text, membership_start_date integer, membership_status text, FOREIGN KEY (person_id) REFERENCES person(id) )"
table,get_fit_now_check_in,get_fit_now_check_in,9,"CREATE TABLE get_fit_now_check_in ( membership_id text, check_in_date integer, check_in_time integer, check_out_time integer, FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id) )"
table,income,income,10,"CREATE TABLE income ( ssn integer PRIMARY KEY, annual_income integer )"
table,solution,solution,11,"CREATE TABLE solution ( user integer, value text )"


In [101]:
%sql SELECT * FROM crime_scene_report where city == 'SQL City' and date = 20180115 and type = 'murder'


 * sqlite:///sql-murder-mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


## Clue 1 
 Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". 
The second witness, named Annabel, lives somewhere on "Franklin Ave".''' 



In [105]:
%%sql
SELECT * FROM person 
WHERE address_street_name LIKE 'Northwestern%' 
AND (address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name LIKE 'Northwestern%')
     OR address_number = (SELECT MIN(address_number) FROM person WHERE address_street_name LIKE 'Northwestern%'))

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
89906,Kinsey Erickson,510019,309,Northwestern Dr,635287661


Could be either these too as there address number largest/smallest and hence living in the 'last house'

In [109]:
%%sql
SELECT * FROM person WHERE name LIKE 'Annabel%' and address_street_name= "Franklin Ave"

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


## At least 2 of the 3 people have information about the murder

In [113]:
%sql SELECT * FROM Interview WHERE person_id in (14887,89906,16371)

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


This probably means 89906 has no idea, it's Morty or Annabel that are the witnesses

Other clues

- Found that man works a membership at the gym and a Get Fit now member, tag start with 48Z licsence plate "H42W"
- Annabel works at the gym and last saw him jan 9th



In [117]:
%sql SELECT * FROM drivers_license WHERE plate_number LIKE '%H42W%'

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
664760,21,71,black,black,male,4H42WR,Nissan,Altima


## Suspect could be from these 3 ids 

In [121]:
%sql SELECT * FROM get_fit_now_member where id LIKE '48Z%'

 * sqlite:///sql-murder-mystery.db
Done.


id,person_id,name,membership_start_date,membership_status
48Z38,49550,Tomas Baisley,20170203,silver
48Z7A,28819,Joe Germuska,20160305,gold
48Z55,67318,Jeremy Bowers,20160101,gold


In [125]:
%sql SELECT * FROM get_fit_now_member WHERE person_id IN (183779, 423327, 664760);

 * sqlite:///sql-murder-mystery.db
Done.


id,person_id,name,membership_start_date,membership_status


In [129]:
%sql SELECT * FROM get_fit_now_check_in WHERE check_in_date = 20180109 and membership_id in ('48Z38','48Z7A','48Z55')

 * sqlite:///sql-murder-mystery.db
Done.


membership_id,check_in_date,check_in_time,check_out_time
48Z7A,20180109,1600,1730
48Z55,20180109,1530,1700


In [131]:
#Either 48Z7A or 48Z55 is the killer, lets see how owns the car with licsense plate with H24W

In [133]:
%sql SELECT * FROM person WHERE license_id in (183779,423327,664760)

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
51739,Tushar Chandra,664760,312,Phi St,137882671
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
78193,Maxine Whitely,183779,110,Fisk Rd,137882671


In [135]:
# The killer is Jeremy Bowers, however he was Hired !

In [137]:
%sql SELECT * FROM Interview WHERE person_id = 67318 

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


In [139]:
# USe clue she attended SQL symphony concert 3 times

In [141]:
%%sql
SELECT * FROM facebook_event_checkin
WHERE event_name = 'SQL Symphony Concert' and date LIKE '%12%'
GROUP BY person_id 
HAVING count(*) = 3

 * sqlite:///sql-murder-mystery.db
Done.


person_id,event_id,event_name,date
24556,1143,SQL Symphony Concert,20171207
99716,1143,SQL Symphony Concert,20171206


In [151]:
%sql SELECT * FROM PERSON WHERE id in (24556,99716)

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
24556,Bryan Pardo,101191,703,Machine Ln,816663882
99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [159]:
%sql SELECT * FROM drivers_license where id in (101191,202298)

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
202298,68,66,green,red,female,500123,Tesla,Model S


In [ ]:
#Mitanda Priestly the master mind